In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import os
import numpy as np
import shutil
import zipfile
from skimage import io, draw

In [ ]:
# paths
train_json = 'insta_100_items.json'
dataset_dir = 'dataset'
images_dir = 'images'
masks_dir = 'masks'

In [ ]:
# create paths
if not os.path.exists(masks_dir):
  os.mkdir(masks_dir)
if not os.path.exists(images_dir):
  os.mkdir(images_dir)
if not os.path.exists(dataset_dir):
  os.mkdir(dataset_dir)

In [ ]:
# delete the dict keys
annotations = list(json.load(open(train_json)).values())  
# skip unannotated images.
annotations = [a for a in annotations if a['regions']]

# create masks
for a in annotations:
    polygons = [r['shape_attributes'] for r in a['regions']]
    image_path = os.path.join(images_dir, a['filename'])
    image = io.imread(image_path)
    height, width = image.shape[:2]
    mask = np.zeros([height, width],dtype=np.uint8)
    for p in polygons:
        if p['name'] == 'polygon':
          rr, cc = draw.polygon(p['all_points_y'], p['all_points_x'])
        elif p['name'] == 'circle':
          rr, cc = draw.circle(p['cx'], p['cy'], p['r'])
        print(mask.shape, image_path)
        for i, j in zip(rr, cc):
          if i >= 0 and j >= 0 and i < mask.shape[0] and j < mask.shape[1]:
            mask[i, j] = 1
    mask = mask.astype(float)
    io.imsave(os.path.join(masks_dir, a['filename']), mask)

(612, 612) images/2013-09-29 09.46.39 555409553222055833_lookingthroughwindow.jpg
(612, 612) images/2013-09-29 09.46.39 555409553222055833_lookingthroughwindow.jpg
(612, 612) images/2013-09-29 09.46.39 555409553222055833_lookingthroughwindow.jpg


In [ ]:
# form the dataset folder
shutil.move(images_dir, dataset_dir)
shutil.move(masks_dir, dataset_dir)
shutil.move(dataset_dir, '/')

'dataset/masks'

In [ ]:
# change folder name!
!zip -r dataset.zip /dataset

updating: dataset/ (stored 0%)
  adding: dataset/images/ (stored 0%)
  adding: dataset/images/2013-09-29 09.46.39 555409553222055833_lookingthroughwindow.jpg (deflated 1%)
  adding: dataset/masks/ (stored 0%)
  adding: dataset/masks/2013-09-29 09.46.39 555409553222055833_lookingthroughwindow.jpg (deflated 55%)


In [ ]:
# move to gdrive
shutil.move('dataset.zip', '/content/drive/My Drive/window_view/datasets')

'/content/drive/My Drive/window_view/datasets/dataset.zip'